In [7]:
import torch
import random

import torch.nn.functional as F
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.utils import standardize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import UpperConfidenceBound
from botorch.optim import optimize_acqf
import torch.nn.functional as F
from botorch.acquisition import ExpectedImprovement



In [12]:
BATCHSIZE = 10
TARGET = [0.34, 0.2]


#r = torch.tensor([[random.uniform(0, 1), random.uniform(0, 1)]], dtype=torch.double)  
def dif(tensor, target):
  target = torch.tensor([target], dtype=torch.double)
  r = target - tensor 
  r = torch.abs(r)
  r = torch.sum(r)
  #r = torch.clamp(r,0,1)
  return 1-torch.tensor([[r]], dtype=torch.double)
  


def getSample():
  return torch.tensor([[random.uniform(0, 1), random.uniform(0, 1)]], dtype=torch.double)  

X = torch.tensor([], dtype=torch.double)
Y = torch.tensor([], dtype=torch.double)
# Create a Standardize transform using the data

for i in range(BATCHSIZE):
  x = getSample()
  X = torch.cat((X, x), dim=0)
  y = dif(x, TARGET)
  Y = torch.cat((Y, y), dim=0)
  
  
Y =  F.normalize(Y, dim=None, p=2)

print(Y)


list(zip(X, Y))


tensor([[ 0.2240],
        [ 0.1372],
        [ 0.5685],
        [ 0.2797],
        [ 0.2248],
        [ 0.1773],
        [ 0.3196],
        [ 0.3792],
        [ 0.4417],
        [-0.0814]], dtype=torch.float64)


[(tensor([0.8764, 0.0169], dtype=torch.float64),
  tensor([0.2240], dtype=torch.float64)),
 (tensor([0.0258, 0.7140], dtype=torch.float64),
  tensor([0.1372], dtype=torch.float64)),
 (tensor([0.1457, 0.2939], dtype=torch.float64),
  tensor([0.5685], dtype=torch.float64)),
 (tensor([0.8145, 0.0246], dtype=torch.float64),
  tensor([0.2797], dtype=torch.float64)),
 (tensor([0.8812, 0.0227], dtype=torch.float64),
  tensor([0.2248], dtype=torch.float64)),
 (tensor([0.6193, 0.6987], dtype=torch.float64),
  tensor([0.1773], dtype=torch.float64)),
 (tensor([0.8468, 0.2930], dtype=torch.float64),
  tensor([0.3196], dtype=torch.float64)),
 (tensor([0.7966, 0.2686], dtype=torch.float64),
  tensor([0.3792], dtype=torch.float64)),
 (tensor([0.0820, 0.3889], dtype=torch.float64),
  tensor([0.4417], dtype=torch.float64)),
 (tensor([0.7904, 0.8515], dtype=torch.float64),
  tensor([-0.0814], dtype=torch.float64))]

In [13]:
train_Y = standardize(Y)
#train_Y = Y
gp = SingleTaskGP(X, train_Y)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_mll(mll)
bounds = torch.stack([torch.zeros(2), torch.ones(2)])

In [16]:

# Define the Expected Improvement (EI) acquisition function
ei = ExpectedImprovement(gp, best_f=torch.max(Y))

# 'best_observed_value' is the best observed value from your optimization process so far
bounds = torch.stack([torch.zeros(2), torch.ones(2)])

# Optimize the EI acquisition function to find the next candidate point
candidate, acq_value = optimize_acqf(
    ei, bounds=bounds, q=1, num_restarts=20, raw_samples=150,
)

print(candidate)

tensor([[0.3217, 0.1949]])
